In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [3]:
graphPath = "/data/graphDFSample"

In [4]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number
from pyspark.sql import Window
from pyspark.sql.functions import sort_array

reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(sort_array(collect_list("user")).alias("users")) \
    .withColumn("users_size", size("users")) \

In [5]:
from pyspark.sql.functions import size

reversedGraph = reversedGraph.filter(size('users')>1)

In [6]:
import itertools
def get_pairs(column_list): 
    return [ el for el in itertools.combinations(column_list,2)]

In [7]:
from pyspark.sql.functions import udf

from pyspark.sql.types import *

schema = ArrayType(StructType([
    StructField("A", IntegerType(), False),
    StructField("B", IntegerType(), False)
]))

udf_pairs = udf(get_pairs, schema)

In [8]:
explodedGraph = reversedGraph.withColumn("pairs", explode(udf_pairs("users")))

In [9]:
graph = explodedGraph.select("friend", "pairs").groupby("pairs").count()

In [10]:
final_graph = graph.select("count",graph.pairs.A.alias("A"), graph.pairs.B.alias("B"))

In [11]:
window = Window.orderBy(col("count").desc())
    
top50 = final_graph.withColumn("row_number", row_number().over(window)) \
            .filter(col("row_number") < 50) \
            .select(col("count"),col("A"),col("B")) \
            .orderBy(col("count").desc()) \
            .collect()

In [12]:
for val in top50:
    print '%s %s %d' % val

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 21864412 31964081
2585 40003405 42973992
2581 27967558 43548989
2579 23848749 31964081
2578 15485897 21864412
2578 27967558 28135661
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985